In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pyFAI
import pyFAI.calibrant
import pyFAI.azimuthalIntegrator
import ipywidgets as widgets
from ipywidgets import interact
import time

In [23]:
def get_detector(det):
    return pyFAI.detector_factory(det)

def get_energy(wavelength):
    return 12.398*wavelength # returned in keV

def get_wavelength(energy):
    return 12.398/energy*1e-10 # returned in m

def get_radians(deg):
    return np.radians(deg)

In [24]:
def get_standard(std, energy):
    if type(std).__module__ == np.__name__:
        # TODO: implement list of 2th or Q in an array
        # rather than a standard
        print('Not yet implemented!')
    elif type(std) == str:
        standard = pyFAI.calibrant.get_calibrant(std)
        standard.set_wavelength(get_wavelength(energy))
    return standard

In [25]:
def geo_cone(X, Y, Z, rota, tilt, yoff, dist):
    # rotate the sample around y
    a = np.deg2rad(tilt) + np.deg2rad(rota)
    t = np.transpose(np.array([X,Y,Z]), (1,2,0))
    m = [[np.cos(a), 0, np.sin(a)],[0,1,0],[-np.sin(a), 0, np.cos(a)]]
    X,Y,Z = np.transpose(np.dot(t, m), (2,0,1))
    # compensate for tilt
    comp = np.deg2rad(tilt) * dist
    return Y,X+comp-yoff,Z

In [26]:
detector = get_detector('eiger2_4m')

In [27]:
detector.getPyFAI()

OrderedDict([('detector', 'Eiger2 4M'),
             ('pixel1', 7.5e-05),
             ('pixel2', 7.5e-05),
             ('max_shape', (2162, 2068))])

In [28]:

#detector = pyFAI.detector_factory("Eiger2 CdTe 4M")
#mask = detector.mask.copy()
#print(detector)

# detector pixel array inspired from pyFAI.geometry.Geometry.calcfrom1d
def plot(det, 
         method='cones', 
         standard='Si', 
         energy=20.5, 
         dist=0.075, 
         offset1=0.08, 
         offset2=0.08, 
         rot=0, 
         tilt=0):
    
    # setup a timer
    t0 = time.perf_counter()
    
    # setup the detector
    detector = get_detector(det)
    print(detector.get_name())
    print(f'Init detector: {time.perf_counter()-t0}')
    
    # init the image
    image = np.zeros(detector.max_shape)
    
    # check if gaps exist and mask them if present
    # TODO: this currently fails for PE as .copy() is
    # not defined
    mask = detector.mask.copy()
    print(f'Init mask: {time.perf_counter()-t0}')
    if mask is not None:
        #image[np.where(mask)] = -1 # dummy value for non-existant pixels
        print(f'Mask out gaps: {time.perf_counter()-t0}')
        
        
    fig, ax = plt.subplots()
    if method == 'pyFAI':        
        std = get_standard(standard, energy)
    
        geom = pyFAI.azimuthalIntegrator.AzimuthalIntegrator(dist=dist, 
                                                         wavelength=get_wavelength(energy),
                                                         poni1=offset1,
                                                         poni2=offset2,
                                                         rot1=get_radians(tilt),
                                                         rot2=get_radians(rot),
                                                         detector=detector
                                                        )
    
        image = std.fake_calibration_image(geom)
        ax.imshow(image, origin='lower')
        ax.imshow(mask, alpha=0.3, origin='lower')    
    elif method == 'cones':
        #ax.imshow(mask, alpha=0.3, origin='lower')   
        # calculate physical sizes of detector in 
        # horizontal (x) and vertical (y) direction
        detx = detector.shape[0]*detector.pixel1
        dety = detector.shape[1]*detector.pixel2
        
        # figure out max dimension
        gridmax = max(detx, dety)
        
        # prepare grids within the detector
        steps = 50 # number of steps in the grid
        xmesh = np.linspace(-gridmax, gridmax, steps)
        ymesh = np.linspace(-gridmax, gridmax, steps)
        
        scale = 1
        X, Y = np.meshgrid(xmesh, ymesh)
        
        #2:_stl*4*np.pi
        Z = np.sqrt(X**2+Y**2)*5 #2.10 in Q
        
        X, Y, Z = geo_cone(X, Y, Z, 0, 0, 0, dist)
        ax.contour(X, Y, Z, [dist], 
                   #colors=plo.cont_orig_color, 
                   #alpha=plo.cont_orig_alpha
                  )
        #ax.imshow(mask, alpha=0.3, origin='lower')   


    

   

    return energy


In [29]:
interact(plot, 
         det=['Eiger2 CdTe 4M', 'Pilatus CdTe 2M', 'Perkin detector'], 
         method=['pyFAI'], # cones commented out
         standard=['Si', 'Ni'],
         energy=(10, 35),
         dist=(0.05, 0.3, 0.001),
         offset1=(0, 0.5, 0.001),
         offset2=(0, 0.5, 0.001),
         rot=(0,45),
         tilt=(0,45)
         )

interactive(children=(Dropdown(description='det', options=('Eiger2 CdTe 4M', 'Pilatus CdTe 2M', 'Perkin detect…

<function __main__.plot(det, method='cones', standard='Si', energy=20.5, dist=0.075, offset1=0.08, offset2=0.08, rot=0, tilt=0)>